# FmKorea 현대차 커뮤니티 크롤링 코드

### 실행순서

1. 1번째 셀 실행 (크롤링을 위한 함수 정의)
2. 2번째 셀 실행 (검색 키워드 "현대차"로 크롤링 실행, df_samsung 데이터프레임 return)
3. 3번째 셀 실행 (검색 키워드 "현차"으로 크롤링 실행, df_samjeon 데이터프레임 return)
4. 4번째 셀 실행 (생성된 df 2개에서 중복되는 게시글 삭제 및 csv 파일생성)

In [ ]:
import time
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
from datetime import datetime
import re

headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/143.0.0.0 Safari/537.36"
    ),
    "Referer": "https://www.fmkorea.com/",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
}

# 날짜 문자열을 정규화(통일)하는 함수
# - "17:27" 처럼 시간만 있으면 오늘 날짜(YYYY-MM-DD)로 치환
# - "2026.01.12" 처럼 점(.)으로 된 날짜는 "-"로 변경
def normalize_date(date_str: str) -> str:
    today = datetime.now().strftime("%Y-%m-%d")
    s = (date_str or "").strip()
    if re.match(r"^\d{1,2}:\d{2}$", s):
        return today
    if "." in s:
        return s.replace(".", "-")
    return s

# 검색 결과 페이지를 순회하며 게시글 리스트를 수집해서 DataFrame으로 반환
# start_page ~ end_page: 수집할 페이지 범위
def crawl_one(session: requests.Session, url_base: str, source_name: str,
              start_page=1, end_page=10, sleep_sec=2) -> pd.DataFrame:
    data = {"탭": [], "제목": [], "글쓴이": [], "날짜": [], "댓글수": [], "조회": [], "추천": [],
            "post_url": [], "source": []}

    for page in range(start_page, end_page + 1):
        url = url_base.format(page)

        r = session.get(url, timeout=20)
        r.raise_for_status()

        soup = bs(r.text, "lxml")
        rows = soup.select("table.bd_lst.bd_tb_lst.bd_tb tbody tr")
        if not rows:
            print(f"[{source_name}] {page}페이지: rows=0 → 중단")
            break

        page_added = 0
        for tr in rows:
            cate_a = tr.select_one("td.cate a")
            title_a = tr.select_one("td.title a.hx")
            author_a = tr.select_one("td.author a")
            reply_a = tr.select_one("td.title a.replyNum")
            time_td = tr.select_one("td.time")
            mno_tds = tr.select("td.m_no")

            if not (cate_a and title_a and author_a and time_td and len(mno_tds) >= 2):
                continue
            reply_cnt = int(reply_a.get_text(strip=True)) if reply_a else 0  # 추가
            
            views = mno_tds[0].text.strip()
            votes = mno_tds[1].text.strip()

            href = title_a.get("href", "")
            post_url = "https://www.fmkorea.com" + href if href.startswith("/") else href

            data["탭"].append(cate_a.get_text(strip=True))
            data["제목"].append(title_a.get_text(" ", strip=True))
            data["글쓴이"].append(author_a.get_text(strip=True))
            data["날짜"].append(normalize_date(time_td.get_text(strip=True)))
            data["조회"].append(int(views.replace(",", "")) if views else 0)
            data["댓글수"].append(reply_cnt)
            data["추천"].append(int(votes.replace(",", "")) if votes else 0)
            data["post_url"].append(post_url)
            data["source"].append(source_name)

            page_added += 1

        print(f"[{source_name}] {page}페이지 완료 / 이번 페이지 {page_added}개 / 누적 {len(data['post_url'])}개")

        if page_added == 0:
            print(f"[{source_name}] {page}페이지: page_added=0 → 중단")
            break

        time.sleep(sleep_sec)

    return pd.DataFrame(data)


## 크롤링 실행

https://www.fmkorea.com/search.php?mid=stock&category=2997203870&listStyle=list&search_keyword=%ED%98%84%EB%8C%80%EC%B0%A8&search_target=title_content&page=1

- 코드 실행시, 위 링크로 접속한뒤에 다시 링크를 복사한뒤 붙여넣어주세요.
- 붙여넣고 링크 맨 뒤, page 부분을 {}로 수정해주세요.
- 코드 실행시, 페이지 새로고침 후 실행해주세요.

In [21]:
# 검색키원드 "현대차"

url_base_hyundai = "https://www.fmkorea.com/search.php?mid=stock&category=2997203870&listStyle=list&search_keyword=%ED%98%84%EB%8C%80%EC%B0%A8&search_target=title_content&page={}"

# Session을 쓰면 동일한 세션(쿠키 등)을 유지한 채 여러 페이지를 연속 요청할 수 있음 
session_hyundai = requests.Session()
session_hyundai.headers.update(headers)  # 세션별 쿠키/헤더 유지

start_page = 401    
end_page = 500
#61~500
# 크롤링 실행
df_hyundai = crawl_one(
    session_hyundai,
    url_base_hyundai,
    "현대차",
    start_page=start_page,
    end_page=end_page
)

df_hyundai.tail()

[현대차] 401페이지 완료 / 이번 페이지 20개 / 누적 20개
[현대차] 402페이지 완료 / 이번 페이지 20개 / 누적 40개
[현대차] 403페이지 완료 / 이번 페이지 20개 / 누적 60개
[현대차] 404페이지 완료 / 이번 페이지 20개 / 누적 80개
[현대차] 405페이지 완료 / 이번 페이지 20개 / 누적 100개
[현대차] 406페이지 완료 / 이번 페이지 20개 / 누적 120개
[현대차] 407페이지 완료 / 이번 페이지 20개 / 누적 140개
[현대차] 408페이지 완료 / 이번 페이지 20개 / 누적 160개
[현대차] 409페이지 완료 / 이번 페이지 20개 / 누적 180개
[현대차] 410페이지 완료 / 이번 페이지 20개 / 누적 200개
[현대차] 411페이지 완료 / 이번 페이지 20개 / 누적 220개
[현대차] 412페이지 완료 / 이번 페이지 20개 / 누적 240개
[현대차] 413페이지 완료 / 이번 페이지 20개 / 누적 260개
[현대차] 414페이지 완료 / 이번 페이지 20개 / 누적 280개
[현대차] 415페이지 완료 / 이번 페이지 20개 / 누적 300개
[현대차] 416페이지 완료 / 이번 페이지 20개 / 누적 320개
[현대차] 417페이지 완료 / 이번 페이지 20개 / 누적 340개
[현대차] 418페이지 완료 / 이번 페이지 20개 / 누적 360개
[현대차] 419페이지 완료 / 이번 페이지 20개 / 누적 380개
[현대차] 420페이지 완료 / 이번 페이지 20개 / 누적 400개
[현대차] 421페이지 완료 / 이번 페이지 20개 / 누적 420개
[현대차] 422페이지 완료 / 이번 페이지 20개 / 누적 440개
[현대차] 423페이지 완료 / 이번 페이지 20개 / 누적 460개
[현대차] 424페이지 완료 / 이번 페이지 20개 / 누적 480개
[현대차] 425페이지 완료 / 이번 페이지 20개 / 누적 500개
[현대차] 426페이지 완료 / 이번 페이지 20개 

,탭,제목,글쓴이,날짜,댓글수,조회,추천,post_url,source
1995,국내주식,"현대자동차-삼성 그룹, '기술 제휴' 전방위로 확산",sadsdl,2025-03-03,0,249,1,https://www.fmkorea.com/index.php?mid=stock&ca...,현대차
1996,국내주식,현대차 주주 긁는법,주식초보응애,2025-03-03,4,518,6,https://www.fmkorea.com/index.php?mid=stock&ca...,현대차
1997,국내주식,현대차 바닥임 나랑 내기해도좋아,내일사라,2025-03-03,11,505,7,https://www.fmkorea.com/index.php?mid=stock&ca...,현대차
1998,국내주식,삼성SDI “ 현대차 기아와 로봇 자율주행차 공동마케팅…‘인셀리전트 라이프’ 혁신 ...,sadsdl,2025-03-03,2,531,1,https://www.fmkorea.com/index.php?mid=stock&ca...,현대차
1999,국내주식,"배터리 동맹' 삼성· 현대차 , 로봇·자율주행차 분야 첫 공동마케팅",sadsdl,2025-03-03,0,203,2,https://www.fmkorea.com/index.php?mid=stock&ca...,현대차


## 크롤링 실행

https://www.fmkorea.com/search.php?mid=stock&category=2997203870&listStyle=list&search_keyword=%ED%98%84%EC%B0%A8&search_target=title_content&page=1

- 코드 실행시, 위 링크로 접속한뒤에 다시 링크를 복사한뒤 붙여넣어주세요.
- 붙여넣고 링크 맨 뒤, page 부분을 {}로 수정해주세요.
- 코드 실행시, 페이지 새로고침 후 실행해주세요.

In [ ]:
# 검색키워드 "현차"

url_base_hyuncha = "https://www.fmkorea.com/search.php?mid=stock&category=2997203870&listStyle=list&search_keyword=%ED%98%84%EC%B0%A8&search_target=title_content&page={}"

# Session을 쓰면 동일한 세션(쿠키 등)을 유지한 채 여러 페이지를 연속 요청할 수 있음 
session_hyuncha = requests.Session()
session_hyuncha.headers.update(headers)  # 세션별 쿠키/헤더 유지
#19~119
start_page = 101
end_page = 119

# 크롤링 실행
df_hyuncha = crawl_one(
    session_hyuncha,
    url_base_hyuncha,
    "현차",
    start_page=start_page,
    end_page=end_page
)

df_hyuncha.tail()

csv 파일 생성

In [22]:
# 두 데이터프레임 합치기

df_all = pd.concat([df_hyundai], ignore_index=True) 
print(f"합치기 전: {len(df_all):,}")

# 중복 제거: URL 기준
df_all = df_all.drop_duplicates(subset=["post_url"], keep="first").reset_index(drop=True) 
print(f"URL 중복 제거 후: {len(df_all):,}")

# 날짜 내림차순 정렬
df_all["날짜_dt"] = pd.to_datetime(df_all["날짜"], errors="coerce")
df_all = (
    df_all.sort_values(by="날짜_dt", ascending=False, na_position="last")
          .drop(columns=["날짜_dt"])
          .reset_index(drop=True)
)

df_all.to_csv("5.csv", index=False, encoding="utf-8-sig")
df_all.tail()

합치기 전: 2,000
URL 중복 제거 후: 2,000


,탭,제목,글쓴이,날짜,댓글수,조회,추천,post_url,source
1995,국내주식,"현대자동차-삼성 그룹, '기술 제휴' 전방위로 확산",sadsdl,2025-03-03,0,249,1,https://www.fmkorea.com/index.php?mid=stock&ca...,현대차
1996,국내주식,현대차 주주 긁는법,주식초보응애,2025-03-03,4,518,6,https://www.fmkorea.com/index.php?mid=stock&ca...,현대차
1997,국내주식,현대차 바닥임 나랑 내기해도좋아,내일사라,2025-03-03,11,505,7,https://www.fmkorea.com/index.php?mid=stock&ca...,현대차
1998,국내주식,삼성SDI “ 현대차 기아와 로봇 자율주행차 공동마케팅…‘인셀리전트 라이프’ 혁신 ...,sadsdl,2025-03-03,2,531,1,https://www.fmkorea.com/index.php?mid=stock&ca...,현대차
1999,국내주식,"배터리 동맹' 삼성· 현대차 , 로봇·자율주행차 분야 첫 공동마케팅",sadsdl,2025-03-03,0,203,2,https://www.fmkorea.com/index.php?mid=stock&ca...,현대차


100페이지씩 나누어진 데이터를 하나의 csv로 결합

In [23]:
import pandas as pd
import glob

# 1) 합칠 CSV 파일들 (현재 폴더의 csv 전부면 이렇게)
files = ["1.csv","2.csv","3.csv","4.csv","5.csv"]

# 2) 모두 읽어서 합치기
dfs = [pd.read_csv(f) for f in files]
df_all = pd.concat(dfs, ignore_index=True)

print("합치기 전:", len(df_all))

# 3) 중복 제거 (post_url 기준이 가장 확실)
df_all = df_all.drop_duplicates(subset=["post_url"], keep="first").reset_index(drop=True)  # [web:60]
print("중복 제거 후:", len(df_all))

# 4) 날짜 내림차순 정렬(안전하게 datetime 변환 후 정렬)
df_all["날짜_dt"] = pd.to_datetime(df_all["날짜"], errors="coerce")
df_all = (
    df_all.sort_values(by="날짜_dt", ascending=False, na_position="last")  # [web:102]
          .drop(columns=["날짜_dt"])
          .reset_index(drop=True)
)

# 5) 최종 저장
df_all.to_csv("fm_hyundai_normal.csv", index=False, encoding="utf-8-sig")

합치기 전: 10820
중복 제거 후: 10820
